<a href="https://colab.research.google.com/github/maegnji/XGBoost/blob/main/221024_ch02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
  from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 경로 설정 (공백있을시에 앞에 \필요)
%cd /content/drive/MyDrive/python/XGBoost/Chapter02

/content/drive/MyDrive/python/XGBoost/Chapter02


In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_census = pd.read_csv('census_cleaned.csv')

In [5]:
x = df_census.iloc[:,:-1]
y = df_census.iloc[:,-1]

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2)

In [7]:
# accuracy_score()는 맞는횟수 / 전체횟수
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [8]:
# 로지스틱회귀분석과 비슷하게 해주기 위하여 random_state=2로 세팅
clf = DecisionTreeClassifier(random_state=2)

In [9]:
clf.fit(x_train, y_train)

DecisionTreeClassifier(random_state=2)

In [10]:
y_pred = clf.predict(x_test)

In [11]:
# 동일한 데이터셋으로 1장에서 로지스틱 회귀의 정확도와 거의 비슷함
accuracy_score(y_pred, y_test)

0.8131679154894976

## 2.4.1 결정 트리 회귀 모델

In [13]:
df_bikes = pd.read_csv('bike_rentals_cleaned.csv')
x_bikes = df_bikes.iloc[:,:-1]
y_bikes = df_bikes.iloc[:,-1]
from sklearn.linear_model import LinearRegression
x_train, x_test, y_train, y_test = train_test_split(x_bikes, y_bikes, random_state=2)

In [16]:
# DecisionTreeRegressor, cross_val_score import
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

In [19]:
# DecisionTreeRegressor 객체 만든 뒤 cross_val_score로 모델 평가
reg = DecisionTreeRegressor(random_state = 2)
scores = cross_val_score(reg, x_bikes, y_bikes, scoring = 'neg_mean_squared_error', cv=5)

In [20]:
rmse = np.sqrt(-scores)
print('RMSE 평균 : %0.2f' % (rmse.mean()))
# RMSE 가 너무 크게나온다! -> 과대적합됐는지 확인 필요!

RMSE 평균 : 1233.36


In [23]:
# 테스트 세트에 대한 예측 전, 훈련 세트의 성능 확인!
# 0이 나온다는건 모든 데이터를 정확하게 맞췄다는 것임!! -> 과대적합!
reg = DecisionTreeRegressor()
reg.fit(x_train, y_train)
y_pred = reg.predict(x_train)
from sklearn.metrics import mean_squared_error
reg_mse = mean_squared_error(y_train, y_pred)
reg_rmse = np.sqrt(reg_mse)
reg_rmse

0.0

In [24]:
# 리프 노드의 샘플 개수를 확인해보자!
# 훈련된 reg객체의 tree_ 속성에 훈련한 트리 객체가 저장되어 있음!
# 전체 샘플은 548개인데, 리프노드가 547이라는 말은 너무 과대적합됨!!
leaf_node_count = 0
tree = reg.tree_
for i in range(tree.node_count):
  if (tree.children_left[i] == -1) and (tree.children_right[i] == -1):
    leaf_node_count += 1
    if tree.n_node_samples[i] > 1:
      print('노드 인덱스 :', i, ', 샘플 개수 :', tree.n_node_samples[i])
print('전체 리프 노드 개수:', leaf_node_count)

노드 인덱스 : 124 , 샘플 개수 : 2
전체 리프 노드 개수: 547
